In [2]:
import sys

import numpy as np
from skimage.transform import rescale, resize
from skimage.filters import gaussian
from cellpose import models
import torch
import tifffile
import nd2

import napari

In [3]:
viewer = napari.Viewer()

In [4]:
if sys.platform == 'darwin':
    d = torch.device('mps')
    model = models.Cellpose(gpu=False, device=d, model_type='cyto2')
else:
    # change gpu=True if on windows, and get rid of device
    model = models.Cellpose(gpu=True, model_type='cyto2')

### nd2 files
We will be using a Nikon file for this notebook, using the `nd2` package. The `nd2` package has an imread function like skimage and tifffile:

```python
data = nd2.imread(filename)
```

Instead, we will use the `ND2File` object gives us access to the metadata (like x, y, z scaling, excitation wavelengths, etc.) that we will need.


In [5]:
imagefile = 'Data/WT003.nd2'

### create the file object - this doesn't read the image array data
nd2file = nd2.ND2File(imagefile)
image = nd2file.asarray()
image.shape

(40, 3, 2044, 2048)

Add the image to the napari viewer. From the shape output in the last cell, the channel axis is 1

In [6]:
viewer.add_image(image, channel_axis=1)

[<Image layer 'Image' at 0x2d723fac0>,
 <Image layer 'Image [1]' at 0x2c42af700>,
 <Image layer 'Image [2]' at 0x2d7c9dd50>]

Rotate the image in 3D and notice the 3D scaling is not set. To get the right values for the z scaling we need to look at the metadata.


### Metadata

Exlpore what is inside the metadata variable of the nd2file object

In [ ]:
nd2file.metadata.channel

In [7]:
md1 = nd2file.metadata.channels[1]
xum, yum, zum = md1.volume.axesCalibration
zscale = xum/zum
xum, yum, zum, zscale

(0.065, 0.065, 0.3, 0.21666666666666667)

### Changing the layer scale

Use a for loop to change the scale of each layer

In [8]:
for y in viewer.layers:
    y.scale = (1/zscale, 1, 1)

### Cellpose 3D

Cellpose has a 3D option in `model.eval`, but it is not really 3D. It does the eval on every xy plane, then every yz plane, then every xz plane. After the eval cellpose reconstructs the results from each plane into 3D masks/labels.  For this to work the scaling in z needs to match the xy scaling. The `anisotropy` parameter can be used, and cellpose will adjust the input image. Another option is to rescale the image before using cellpose. 

In [9]:
scaled = rescale(image, (1, 1, zscale, zscale), preserve_range=True)
scaled = gaussian(scaled, sigma=(1, 0, 2, 2))
scaled.shape

(40, 3, 443, 444)

In [10]:
viewer.layers.clear()
viewer.add_image(scaled, channel_axis=1)

[<Image layer 'Image' at 0x2cdec6e30>,
 <Image layer 'Image [1]' at 0x2cfafbf10>,
 <Image layer 'Image [2]' at 0x35d8bc940>]

In [11]:
masks, _, _, _ = model.eval(scaled, diameter=75, do_3D=True, channels=[1,2],
                            cellprob_threshold=1,
                            flow_threshold=.3) 

In [12]:
viewer.add_labels(masks)

<Labels layer 'masks' at 0x35dd87250>

Traceback (most recent call last):
  File "/Users/cjw/mambaforge/envs/py310/lib/python3.10/site-packages/napari/components/viewer_model.py", line 415, in _update_status_bar_from_cursor
    self.status = active.get_status(
  File "/Users/cjw/mambaforge/envs/py310/lib/python3.10/site-packages/napari/layers/labels/labels.py", line 1391, in get_status
    value = self.get_value(
  File "/Users/cjw/mambaforge/envs/py310/lib/python3.10/site-packages/napari/layers/base/base.py", line 1103, in get_value
    value = self._get_value_3d(
  File "/Users/cjw/mambaforge/envs/py310/lib/python3.10/site-packages/napari/layers/labels/labels.py", line 1048, in _get_value_3d
    self._get_value_ray(
  File "/Users/cjw/mambaforge/envs/py310/lib/python3.10/site-packages/napari/layers/labels/labels.py", line 1016, in _get_value_ray
    values = im_slice[tuple(clamped.T)]
IndexError: index 57 is out of bounds for axis 1 with size 40
Traceback (most recent call last):
  File "/Users/cjw/mambaforge/envs/py310/l

To use the cellpose results with the original image, the masks need to be scaled back to the original size. The `order` parameter is the key to making this successful.
Settting `order=0` makes `resize` use nearest neighbors when upscaling an image rather than interpolation. 

In [14]:
shape = (image.shape[0], image.shape[2], image.shape[3])
smasks = resize(masks, shape, order=0, preserve_range=True)

In [15]:
viewer.layers.clear()
viewer.add_image(image, channel_axis=1)
viewer.add_labels(smasks)


<Labels layer 'smasks' at 0x2cfafb5b0>

In [ ]:
for y in viewer.layers:
    y.scale = (1/zscale, 1, 1)

In [16]:
smasks.shape

(40, 2044, 2048)

In [17]:
640*512*5564*3/1e9

5.46963456